# Reducción del dataset

En la fase 1 del proyecto (Identificación y estudio teórico de datos industriales) analizamos la estructura del dataset *Tennessee Eastman Process (TEP)*. Este esta compuesto por simulaciones industriales que representan tanto el comportamiento normal como los 20 distintos tipos de fallos.

En ese análisis detallamos las variables que forman parte del proceso, comprobamos que no había valores nulos, y confirmamos que el volumen de datos del dataset era enorme (cientos de miles de registros), especialmente para los archivos de fallos.

Este tamaño es ideal para realizar una investigación avanzada, pero para desarrollar nuestro proyecto, nos supone grandes limitaciones. Por ejemplo, usar el dataste completo afecta en los tiempos de carga de los archivos, retrasa el entrenamiento y dificulta la experimentación del usuario en los paneles.

Por estas razones, antes de avanzar con los siguientes pasos del proyecto, entre ellos realizar un análisis exploratorio más profundo, la ingeniería de características y creación de modelos, vamos a **reducir el dataset original**.

#### Estrategia para la reducción

Para conservar el proceso industrial pero sin usar todas las simulaciones del dataset, vamos a usar un **muestreo estratificado** que se basa en simulaciones completas.

Mantenemos todas la columnas del conjunto de datos original y los segmentos temporales (conservamos las simulaciones completas). Creamos una nueva variable binaria, *fault_present*,que nos puede resultar útil en proximas tareas. En cambio, para cada tipo de fallo seleccionamos 10 simulaciones completas, y para los archivos donde no tenemos fallos, seleccionames 20 simulaciones.

De esta forma garantizamos que los datos reducidos mantienen la dimensionalidad original (las 52 variables del proceso) y reflejamos ambas condiciones, las normales y con los fallos simulados.

### Faulty_Training y Faulty_Testing

Comenzamos reduciendo los archivos que tienen las simulaciones con fallos.

Primero cargamos los archivos y convertimos las columnas numéricas a float, despues creamos una etiqueta para indicar si hay fallo o no. El siguiente paso es eleguir un número limitado de simulaciones por fallo y tambien algunas simulaciones sin fallo para generar los nuevos archivos.

In [2]:
import pyreadr
import pandas as pd
import numpy as np
import os

def load_and_reduce(file_path, n_sims_per_fault=10, n_sims_faultfree=20):
    """
    Carga un archivo .RData, convierte a float32 para ahorrar memoria,
    aplica estratificación por tipo de fallo y devuelve un DataFrame reducido
    manteniendo todas las columnas y sus contenidos.
    """
    try:
        result = pyreadr.read_r(file_path)
        df_name = list(result.keys())[0]
        df = result[df_name]

        # Reducir tipo de dato (float64 -> float32)
        for col in df.select_dtypes(include="float64").columns:
            df[col] = df[col].astype("float32")

        # Añadir etiqueta binaria (sin eliminar nada)
        if 'faultNumber' in df.columns:
            df['fault_present'] = np.where(df['faultNumber'] > 0, 1, 0)

        # Detectar columna de simulación
        sim_col = None
        for candidate in ['simulationRun', 'simRun', 'run']:
            if candidate in df.columns:
                sim_col = candidate
                break
        if sim_col is None:
            raise KeyError("No encontré columna de simulación en el dataset")

        # --- Estratificación ---
        if 'faultNumber' in df.columns:
            # Simulaciones con fallo
            df_faulty = df[df['faultNumber'] > 0]
            selected_faulty_sims = (
                df_faulty.groupby('faultNumber')[sim_col]
                .apply(lambda sims: sims.drop_duplicates().sample(
                    n=min(n_sims_per_fault, len(sims)), random_state=42))
                .reset_index(drop=True)
            )

            # Simulaciones sin fallo
            df_faultfree = df[df['faultNumber'] == 0]
            selected_faultfree_sims = (
                df_faultfree[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df_faultfree[sim_col].unique())), random_state=42)
            )

            # Combinar IDs de simulaciones seleccionadas
            selected_sims = pd.concat([selected_faulty_sims, selected_faultfree_sims])
            df_reduced = df[df[sim_col].isin(selected_sims)]
        else:
            # Caso FaultFree: solo muestreo de simulaciones
            selected_sims = (
                df[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df[sim_col].unique())), random_state=42)
            )
            df_reduced = df[df[sim_col].isin(selected_sims)]

        return df_reduced, sim_col

    except Exception as e:
        print(f"Error al cargar {file_path}: {e}")
        return pd.DataFrame(), None


# Ejemplo de uso completo
DATA_DIR = os.getcwd()
DEST_DIR = os.path.join(DATA_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

file_paths = {
    "Faulty Training": os.path.join(DATA_DIR, "TEP_Faulty_Training.RData"),
    "Faulty Testing": os.path.join(DATA_DIR, "TEP_Faulty_Testing.RData"),
}

for name, path in file_paths.items():
    df_reduced, sim_col = load_and_reduce(path)
    if not df_reduced.empty:
        out_file = os.path.join(DEST_DIR, f"{name.replace(' ', '_')}_reduced.csv")
        df_reduced.to_csv(out_file, index=False)
        print(f"\n--- {name} reducido ---")
        print("Archivo guardado en:", out_file)
        print("Simulaciones:", df_reduced[sim_col].nunique())
        print("Muestras:", len(df_reduced))
        print("Columnas:", df_reduced.columns.tolist())

print("\nTodos los datasets reducidos guardados en la carpeta DatasetReducido.")



--- Faulty Training reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\Faulty_Training_reduced.csv
Simulaciones: 10
Muestras: 100000
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']

--- Faulty Testing reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\Faulty_Testing_reduced.csv
Simulaciones: 

### FaultFree_Training y FaultFree_Testing

Para estos otros dos archivos que no contienen fallos usamos la misma lógica. Cargamos los datasets, convertimos los datos numéricos a float y eleguimos un número limitado de simulaciones. De esta forma creamos un dataset reducido conservando las columnas originales.

In [4]:
import pyreadr
import pandas as pd
import numpy as np
import os

def load_and_reduce(file_path, n_sims_per_fault=10, n_sims_faultfree=20):
    """
    Carga un archivo .RData, convierte a float32 para ahorrar memoria,
    aplica estratificación por tipo de fallo y devuelve un DataFrame reducido
    manteniendo todas las columnas y sus contenidos.
    """
    try:
        result = pyreadr.read_r(file_path)
        df_name = list(result.keys())[0]
        df = result[df_name]

        # Reducir tipo de dato (float64 -> float32)
        for col in df.select_dtypes(include="float64").columns:
            df[col] = df[col].astype("float32")

        # Añadir etiqueta binaria (sin eliminar nada)
        if 'faultNumber' in df.columns:
            df['fault_present'] = np.where(df['faultNumber'] > 0, 1, 0)

        # Detectar columna de simulación
        sim_col = None
        for candidate in ['simulationRun', 'simRun', 'run']:
            if candidate in df.columns:
                sim_col = candidate
                break
        if sim_col is None:
            raise KeyError("No encontré columna de simulación en el dataset")

        # --- Estratificación ---
        if 'faultNumber' in df.columns:
            # Simulaciones con fallo
            df_faulty = df[df['faultNumber'] > 0]
            selected_faulty_sims = (
                df_faulty.groupby('faultNumber')[sim_col]
                .apply(lambda sims: sims.drop_duplicates().sample(
                    n=min(n_sims_per_fault, len(sims)), random_state=42))
                .reset_index(drop=True)
            )

            # Simulaciones sin fallo
            df_faultfree = df[df['faultNumber'] == 0]
            selected_faultfree_sims = (
                df_faultfree[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df_faultfree[sim_col].unique())), random_state=42)
            )

            # Combinar IDs de simulaciones seleccionadas
            selected_sims = pd.concat([selected_faulty_sims, selected_faultfree_sims])
            df_reduced = df[df[sim_col].isin(selected_sims)]
        else:
            # Caso FaultFree: solo muestreo de simulaciones
            selected_sims = (
                df[sim_col]
                .drop_duplicates()
                .sample(n=min(n_sims_faultfree, len(df[sim_col].unique())), random_state=42)
            )
            df_reduced = df[df[sim_col].isin(selected_sims)]

        return df_reduced, sim_col

    except Exception as e:
        print(f"Error al cargar {file_path}: {e}")
        return pd.DataFrame(), None


# ------------------------------
# Ejemplo de uso completo
# ------------------------------
DATA_DIR = os.getcwd()
DEST_DIR = os.path.join(DATA_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

file_paths = {
    "FaultFree Training": os.path.join(DATA_DIR, "TEP_FaultFree_Training.RData"),
    "FaultFree Testing": os.path.join(DATA_DIR, "TEP_FaultFree_Testing.RData"),
}

for name, path in file_paths.items():
    df_reduced, sim_col = load_and_reduce(path)
    if not df_reduced.empty:
        out_file = os.path.join(DEST_DIR, f"{name.replace(' ', '_')}_reduced.csv")
        df_reduced.to_csv(out_file, index=False)
        print(f"\n--- {name} reducido ---")
        print("Archivo guardado en:", out_file)
        print("Simulaciones:", df_reduced[sim_col].nunique())
        print("Muestras:", len(df_reduced))
        print("Columnas:", df_reduced.columns.tolist())

print("\nTodos los datasets reducidos guardados en la carpeta DatasetReducido.")


--- FaultFree Training reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\FaultFree_Training_reduced.csv
Simulaciones: 20
Muestras: 10000
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']

--- FaultFree Testing reducido ---
Archivo guardado en: C:\Users\miren\Documents\GitHub\proyecto_analitica\DatasetReducido\FaultFree_Testing_reduced.csv
Sim

Una vez hemos realizado este proceso para los cuatro archivos del dataste TEP, obtenemos un conjunto de datos reducido, pero representativo y eficiente. Este dataset nuevo será el que usaremos para el resto del proyecto.